Find the colab notebook here <br> <br>

<a href="https://colab.research.google.com/drive/14YtgnCpojbUdVs8CrDR4Hcsyges_HVhj?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before Running
- Set the Runtime type to your preffered GPU
- Run everything under "Run Server" Section to get the url
- Run "Stop Server" to stop server and Tunnel

# Run Server

This will setup everything needed for the backend. The runtime will restart. After restart. Do not run this section again. Run the next section

## Clone the Repo

In [ ]:
!rm -rf qbtrain-be
!git clone https://github.com/pavanreddyml/qbtrain-be.git

Install the `qbtrain` library

In [ ]:
!pip install qbtrain

## Install Ollama

In [ ]:
!sudo apt update && sudo apt install pciutils lshw
!curl -fsSL https://ollama.com/install.sh | sh

## Install Cloudflare

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 \
  -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
!cloudflared --version

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

# Post Install Setup

In [ ]:
from IPython.display import clear_output

## Run Ollama Server

Run Ollama Server

In [ ]:
!nohup sh -c 'while true; do ollama serve; sleep 5; done' > ollama.log 2>&1 &

Pull gemma2:2b

In [ ]:
!ollama pull "gemma2:2b"

## Setup Database Tables for Server

In [ ]:
!python qbtrain-be/python/qbtrainserver/manage.py makemigrations
!python qbtrain-be/python/qbtrainserver/manage.py migrate

## Run QBTrain Server

In [ ]:
# Kill any existing server
!pkill -f gunicorn || true
clear_output()

# Start server in background, capture logs
!nohup gunicorn qbtrainserver.wsgi:application \
  --chdir qbtrain-be/python/qbtrainserver \
  --pythonpath qbtrain/python/qbtrainserver \
  --bind 0.0.0.0:8000 \
  --workers 2 \
  --threads 2 \
  >/content/gunicorn.log 2>&1 &


## Create a Tunnel

In [ ]:
# Kill any existing tunnel
!pkill -f cloudflared || true
clear_output()

# Start tunnel in background, capture logs
!nohup cloudflared tunnel \
  --url http://127.0.0.1:8000 \
  --no-autoupdate \
  --loglevel info \
  --logfile /content/cloudflared.log \
  >/content/cloudflared.out 2>&1 &

!sleep 10
!grep -o 'https://[^ ]*trycloudflare.com' /content/cloudflared.out | head -n 1


# Print Backend Link

Paste this in the app


In [ ]:
!grep -o 'https://[^ ]*trycloudflare.com' /content/cloudflared.out | head -n 1

# Kill Server

## Kill QBTrian Server

In [ ]:
# !pkill gunicorn

## Kill Tunnel

In [ ]:

# !pkill cloudflared

---
